### **Librerias Necesarias**

In [1]:
%run Proyecto_idDrRecursos.ipynb

**Obtención de las frecuencias por las cuales se haran las pruebas**

In [6]:
i = 3000
freqs = []
while i >= 5:
    freqs.append(i)
    i //= 2
freqs

[3000, 1500, 750, 375, 187, 93, 46, 23, 11, 5]

**Función para realizar la prueba i y dentro de si el algorit kNN**

In [3]:
def pruebaI(k_s = 3, perc = 30) -> tuple:
    percent_query = perc
    k = k_s
    #Datos ------------------------------------------
    #Query
    nume_samples_query = int((len(Freqs_drone) * percent_query)/100)                    #Numero de muestras con respecto al porcentanje         
    query_index_samples = random.sample(list(range(0,400)),nume_samples_query)          #Escogemos los indices de query aleatoriamente sin remplazo
    query_index_samples.sort()                                                          #Ordenamos 
    query_samples = Freqs_drone.iloc[query_index_samples]                               #seleccionamos de data_num con respecto a los indices encontrados 
    #Training
    #Agregar inplace como True para eliminar de la original 
    training_samples = Freqs_drone.drop(query_index_samples)                            #Escogemos los indices faltantes

    #Label 
    query_samples_label = label_drone.iloc[query_index_samples]                         #iloc -> [] usando los indices antes encontrados para query
    training_samples_label = label_drone.drop(query_index_samples)                      #Escogemos los indices faltantes 

    #Elementos en total de la query
    totalQ = len(query_samples)
    totalT = len(training_samples)
    size_q  = len(query_samples)                                                        #Size de query
    k_DI = []  
    cont_Error = 0
    #Algoritmo kNN --------------------------------                                     #Aqui se almacenan las k distancias mas pequeñas                                                                          #
    for j in range(size_q):                                                             #iteramos por todos los elementos de la query
        Q = np.tile(query_samples.iloc[j,:],(training_samples.shape[0],1))              #Igual que 'repmat'
        Z = Q - training_samples                                                        
        
        S_with_garbage = Z @ Z.T                                                        #puede usar tambien np.dot(m1,m2)
        distancia_S = np.diag(S_with_garbage) ** (1 / 2)                                #Obtener la diagonal y elevar sus elem a 1/2
        
        sort_index = np.argsort(distancia_S)                                            #Obtener los indices ordenados
        
        k_index = sort_index[:k]                                                        #Seleccionamos los k primeros
        k_dis = distancia_S[k_index]                                                    #Almacenamos las k distancias minimas

        mode_k_dis = mode(k_dis)[0]                                                     #mode retorna (moda, repeticiones)
        index_mode = np.where(distancia_S == mode_k_dis)[0][0]                          #buscamos indice en distancia_S de mode_k_dis
        k_DI.append(index_mode) 
        if query_samples_label.iloc[j, 0] != training_samples_label.iloc[index_mode, 0]:
            cont_Error += 1

    percent_success = (totalQ - cont_Error) * (100 / totalQ)

    return percent_query, totalQ, totalT, k, cont_Error, percent_success, 

**Extracción de datos** 

In [4]:
for f in freqs:
    #Datos globales
    Freqs_drone = pd.read_json(f"Seleccion/Frecuencias/colFreqs{f}.json")
    label_drone = pd.read_csv(f"Seleccion/Frecuencias/labelFreqs{f}.csv")
    for pe_ in range(20, 61, 5):
        for k in [3, 5]:
            data = []
            for _ in range(1000):
                pq, tQ, tT, k, e, p = pruebaI(k_s= k,perc= pe_)
                data.append([pq, tQ, tT,k , e, p])

            data_DF = pd.DataFrame(data, columns=["Porcentaje Query", "Total Query", "Total Training", "k", "Error", "Porcentaje"])
            ruta = f"Seleccion/PruebaPorcentajef{f}/porcentajeExitok{k}p{pe_}f{f}.csv"
            data_DF.to_csv(ruta, index=False)